<a href="https://colab.research.google.com/github/Joycechidi/Deep-Learning-/blob/master/CNN/Malaria_Cell_Detection_Using_VGGNet_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Transfer Learning
In this notebook, I'll be using VGGNet trained on the ImageNet dataset as a feature extractor. 

I'm using VGGNet because it's simple and has great performance, coming in second in the ImageNet competition. 

All the convolutional layers will be kept but I will replace the final fully-connected layer with my own classifier. This way I can use VGGNet as a fixed feature extractor for my images then easily train a simple classifier on top of that.

*   Use all but the last fully-connected layer as a fixed feature extractor.
*   Define a new, final classification layer and apply it to a task of our choice!

I am trying to use the dataset containing infected and uninfected malaria cell images. Downloaded from Kaggle here [https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria/download]

### Objective
Build a convolutional neural network that can detect the presence of malaria parasite in blood cells.

After training this dataset with VGGNet, I will also train with ResNet pre-trained model and then compare the performance between the two models.

In [0]:
import os 
import numpy as np
import torch

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt


%matplotlib inline

In [5]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...') 

CUDA is available!  Training on GPU ...


## **Configuring the directories and spliting the dataset**

In [0]:
# initialize the path to the original input directory of images
data_dir = os.path.join("/content/gdrive/My Drive/Udacity_AI_Codes/DLND_Works/data/")

#initialize the base path to the new directory that will contain the images 
#after computing the training and testing split
base_path = "malaria"

#create the directories for training, validation, and testing
train_path = os.path.sep.join([base_path, "training"])
val_path = os.path.sep.join([base_path, "validation"])
test_path = os.path.sep.join([base_path, "testing"])

#define the amount of data to be used for training 
train_split = 0.8

#the amount of validation data will be 10% of the training data
val_split = 0.1
